In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN, eval_nn, train_mlp, test_mlp, train_mlpbeta, test_mlpbeta, train_cnn, test_cnn
import session_new as session

import process_data as prd
from lda import train_lda, predict, eval_lda, eval_lda_ch

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
sub_type = 'AB'
with open('train_data_raw_'  + sub_type + '.p', 'rb') as f:
    raw, params,_,_ = pickle.load(f)

n_subs = np.max(params[:,0])

In [4]:
mlp = MLP()
mlp_beta = MLPbeta()
cnn = CNN()

mlp(x_train_mlp)
mlp_beta(x_train_mlp)
cnn(x_train_cnn)

mlp.save_weights('mlp_init_2.h5')
mlp_beta.save_weights('mlpbeta_init_2.h5')
cnn.save_weights('cnn_init_2.h5')

In [3]:
train_sess = {'sub_type':sub_type,'train':'fullallmix4', 'train_grp':2, 'train_scale':5, 'cv_type':'all','scaler_load':False,'feat_type':'feat','epochs':30}
test_sess = {'test_grp':4, 'test':'partposrealmixeven24'}
sess = session.Sess(**train_sess)
sess.update(**test_sess)
n_subs = 4#np.max(params[:,0])

for sub in range(2,n_subs+1):
    sess.sub = sub
    # tf.keras.backend.clear_session()

    if sub > 2:
        mlp.load_weights('mlp_init.h5')
        mlp_beta.load_weights('mlpbeta_init.h5')
        cnn.load_weights('cnn_init.h5')
    else:
        # Train NNs
        mlp = MLP()
        mlp_beta = MLPbeta()
        cnn = CNN()

        loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        optimizer = tf.keras.optimizers.Adam()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

        test_loss = tf.keras.metrics.Mean(name='test_loss')
        test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

    train_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.train_grp)
    if np.sum(train_ind):
        trainmlp, validmlp, traincnn, validcnn, y_train, y_valid, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, y_train_aug = prd.prep_train_data(sess,raw,params)
        with open('noisedata_all_emgscalelim/AB' + str(sub) + '_grp_2_fullallmix4_5.p','rb') as f:
            scaler, x_train_cnn, _, x_valid_cnn, _, y_train, y_valid, x_train_lda, y_train_lda, x_train_aug, y_train_aug = pickle.load(f)
        
        x_train_mlp = x_train_cnn.reshape(x_train_cnn.shape[0],-1)
        x_valid_mlp = x_valid_cnn.reshape(x_valid_cnn.shape[0],-1)
        
        trainmlp = tf.data.Dataset.from_tensor_slices((x_train_mlp, y_train)).batch(128)
        validmlp = tf.data.Dataset.from_tensor_slices((x_valid_mlp, y_valid)).batch(128)
        traincnn = tf.data.Dataset.from_tensor_slices((x_train_cnn, y_train)).batch(128)
        validcnn = tf.data.Dataset.from_tensor_slices((x_valid_cnn, y_valid)).batch(128)
        
        sess.scaler = scaler

        models = ['mlp','mlpbeta','cnn']

        for model in models:
            print('Training ' + model)
            for epoch in range(sess.epochs):
                # Reset the metrics at the start of the next epoch
                train_loss.reset_states()
                train_accuracy.reset_states()
                test_loss.reset_states()
                test_accuracy.reset_states()

                # Train MLP
                if model == 'mlp':
                    for x, y in trainmlp:
                        train_mlp(x, y, mlp, loss_fn, optimizer, train_loss, train_accuracy)
                    for x_test, y_test in validmlp:
                        test_mlp(x_test, y_test, mlp, loss_fn, test_loss, test_accuracy)
                # Train MLP Beta
                elif model == 'mlpbeta':
                    for x, y in trainmlp:
                        train_mlpbeta(x, y, mlp_beta, loss_fn, optimizer, train_loss, train_accuracy)
                    for x_test, y_test in validmlp:
                        test_mlpbeta(x_test, y_test, mlp_beta, loss_fn, test_loss, test_accuracy)
                # Train CNN
                elif model == 'cnn':
                    for x, y in traincnn:
                        train_cnn(x, y, cnn, loss_fn, optimizer, train_loss, train_accuracy)
                    for x_test, y_test in validcnn:
                        test_cnn(x_test, y_test, cnn, loss_fn, test_loss, test_accuracy)

                if epoch == 0 or epoch == sess.epochs-1:
                    print(
                        f'Epoch {epoch + 1}, '
                        f'Loss: {train_loss.result():.2f}, '
                        f'Accuracy: {train_accuracy.result() * 100:.2f}, '
                        f'Test Loss: {test_loss.result():.2f}, '
                        f'Test Accuracy: {test_accuracy.result() * 100:.2f}'
                    )
        
        # Train aligned
        y_train_aligned = np.argmax(y_train, axis=1)[...,np.newaxis]

        mlp_enc = mlp.get_layer(name='enc')
        w_mlp, c_mlp,_, _, _ = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aligned)

        mlpbeta_enc = mlp_beta.get_layer(name='enc')
        w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlpbeta_enc(x_train_mlp).numpy(),y_train_aligned)
        
        cnn_enc = cnn.get_layer(name='enc')
        temp = cnn_enc(x_train_cnn[:x_train_cnn.shape[0]//2,...]).numpy()
        temp2 = np.vstack((temp,cnn_enc(x_train_cnn[x_train_cnn.shape[0]//2:,...]).numpy()))
        w_cnn, c_cnn,_, _, _ = train_lda(temp2,y_train_aligned)

        # Train LDA
        w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)
        w_aug,c_aug, _, _, _ = train_lda(x_train_aug,y_train_aug)

        # Load test data
        with open('real_noise/all_real_noise.p', 'rb') as f:
            real_noise_temp, _ = pickle.load(f)

        test_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.test_grp)

        if np.sum(test_ind):
            x_test_cnn, x_test_mlp, x_test_lda, y_test, clean_size = prd.prep_test_data(sess, raw, params, real_noise_temp)
            with open('testdata_all_emgscalelim_noisescalelim/AB' + str(sub) + '_grp_2_partposrealmixeven14_1.p','rb') as f:
                x_test_cnn, _, x_test_lda, y_test = pickle.load(f) 
            x_temp = np.transpose(x_test_lda.reshape((x_test_lda.shape[0],4,-1)),(0,2,1))[...,np.newaxis]
            x_test_cnn = scaler.transform(x_temp.reshape(x_temp.shape[0]*x_temp.shape[1],-1)).reshape(x_temp.shape)

            # Reshape for nonconvolutional SAE
            x_test_mlp = x_test_cnn.reshape(x_test_cnn.shape[0],-1)
            # Test
            mlp_test_aligned = mlp_enc(x_test_mlp).numpy()
            mlpbeta_test_aligned = mlpbeta_enc(x_test_mlp).numpy()
            cnn_test_aligned = cnn_enc(x_test_cnn).numpy()
            y_test_aligned = np.argmax(y_test, axis=1)[...,np.newaxis]

            clean_lda = eval_lda(w, c, x_test_lda[:clean_size,...], y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w, c, x_test_lda[clean_size:,...], y_test_aligned[clean_size:,...])
            print(
                f'LDA ---- '
                f'Clean: {clean_lda * 100:.2f}, '
                f'Noisy: {noisy_lda * 100:.2f}'
            )

            clean_lda = eval_lda(w_aug, c_aug, x_test_lda[:clean_size,...], y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w_aug, c_aug, x_test_lda[clean_size:,...], y_test_aligned[clean_size:,...])
            print(
                f'AUG ---- '
                f'Clean: {clean_lda * 100:.2f}, '
                f'Noisy: {noisy_lda * 100:.2f}'
            )

            clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test,mlp,clean_size)
            clean_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
            print(
                f'MLP ---- '
                f'Clean: {clean_acc * 100:.2f}, '
                f'Noisy: {noisy_acc * 100:.2f}, '
                f'LDA Clean: {clean_lda * 100:.2f}, '
                f'LDA Noisy: {noisy_lda * 100:.2f}'
            )

            clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test,mlp_beta,clean_size)
            clean_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
            print(
                f'MLPB ---- '
                f'Clean: {clean_acc * 100:.2f}, '
                f'Noisy: {noisy_acc * 100:.2f}, '
                f'LDA Clean: {clean_lda * 100:.2f}, '
                f'LDA Noisy: {noisy_lda * 100:.2f}'
            )

            clean_acc, noisy_acc = eval_nn(x_test_cnn, y_test,cnn,clean_size)
            clean_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
            print(
                f'CNN ---- '
                f'Clean: {clean_acc * 100:.2f}, '
                f'Noisy: {noisy_acc * 100:.2f}, '
                f'LDA Clean: {clean_lda * 100:.2f}, '
                f'LDA Noisy: {noisy_lda * 100:.2f}'
            )
        else:
            print('no testing data')
    else:
        print('no training data')



Loading training data: traindata_all/AB2_traindata_2.p
Training mlp

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 1.94, Accuracy: 18.12, Test Loss: 1.90, Test Accuracy: 23.00
Epoch 30, Loss: 1.61, Accuracy: 55.68, Test Loss: 1.59, Test Accuracy: 56.81
Training mlpbeta

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend

In [ ]:
Loading training data: traindata_all/TR1_traindata_2.p
Training mlp
Epoch 1, Loss: 1.89, Accuracy: 20.88, Test Loss: 1.83, Test Accuracy: 30.49
Epoch 30, Loss: 1.57, Accuracy: 59.02, Test Loss: 1.56, Test Accuracy: 60.24
Training mlpbeta
Epoch 1, Loss: 1.79, Accuracy: 35.47, Test Loss: 1.72, Test Accuracy: 41.67
Epoch 30, Loss: 1.46, Accuracy: 70.65, Test Loss: 1.44, Test Accuracy: 72.68
Training cnn
Epoch 1, Loss: 1.74, Accuracy: 41.01, Test Loss: 1.60, Test Accuracy: 57.08
Epoch 30, Loss: 1.41, Accuracy: 75.42, Test Loss: 1.43, Test Accuracy: 73.17
Loading training data: traindata_all/TR1_traindata_4.p
LDA ---- Clean: 63.46, Noisy: 24.32
AUG ---- Clean: 43.89, Noisy: 40.93
MLP ---- Clean: 49.25, Noisy: 42.11, LDA Clean: 55.54, LDA Noisy: 44.25
MLPB ---- Clean: 66.71, Noisy: 54.79, LDA Clean: 61.93, LDA Noisy: 50.71
CNN ---- Clean: 62.29, Noisy: 51.64, LDA Clean: 63.25, LDA Noisy: 51.64
Loading training data: traindata_all/TR2_traindata_2.p
Training mlp
Epoch 1, Loss: 1.95, Accuracy: 16.15, Test Loss: 1.94, Test Accuracy: 17.52
Epoch 30, Loss: 1.62, Accuracy: 54.85, Test Loss: 1.57, Test Accuracy: 58.52
Training mlpbeta
Epoch 1, Loss: 1.82, Accuracy: 32.19, Test Loss: 1.71, Test Accuracy: 43.29
Epoch 30, Loss: 1.49, Accuracy: 67.88, Test Loss: 1.46, Test Accuracy: 70.54
Training cnn
Epoch 1, Loss: 1.83, Accuracy: 33.09, Test Loss: 1.69, Test Accuracy: 49.10
Epoch 30, Loss: 1.52, Accuracy: 64.43, Test Loss: 1.51, Test Accuracy: 65.65
Loading training data: traindata_all/TR2_traindata_4.p
LDA ---- Clean: 84.75, Noisy: 46.36
AUG ---- Clean: 63.57, Noisy: 50.82
MLP ---- Clean: 64.75, Noisy: 58.39, LDA Clean: 57.61, LDA Noisy: 54.61
MLPB ---- Clean: 66.00, Noisy: 61.18, LDA Clean: 77.39, LDA Noisy: 68.86
CNN ---- Clean: 65.79, Noisy: 62.96, LDA Clean: 70.86, LDA Noisy: 64.46

In [ ]:
Loading training data: traindata_manual/AB1_traindata_2.p
Training mlp
Epoch 1, Loss: 1.92, Accuracy: 20.06, Test Loss: 1.86, Test Accuracy: 26.00
Epoch 30, Loss: 1.64, Accuracy: 52.17, Test Loss: 1.79, Test Accuracy: 36.86
Training mlpbeta
Epoch 1, Loss: 1.76, Accuracy: 40.00, Test Loss: 1.73, Test Accuracy: 45.56
Epoch 30, Loss: 1.51, Accuracy: 65.74, Test Loss: 1.57, Test Accuracy: 59.14
Training cnn
Epoch 1, Loss: 1.77, Accuracy: 38.78, Test Loss: 1.73, Test Accuracy: 42.92
Epoch 30, Loss: 1.31, Accuracy: 86.21, Test Loss: 1.43, Test Accuracy: 72.92
no testing data
Loading training data: traindata_manual/AB2_traindata_2.p
Training mlp
Epoch 1, Loss: 1.94, Accuracy: 15.98, Test Loss: 1.94, Test Accuracy: 15.81
Epoch 30, Loss: 1.66, Accuracy: 50.40, Test Loss: 1.68, Test Accuracy: 46.89
Training mlpbeta
Epoch 1, Loss: 1.90, Accuracy: 22.63, Test Loss: 1.86, Test Accuracy: 27.02
Epoch 30, Loss: 1.42, Accuracy: 74.50, Test Loss: 1.47, Test Accuracy: 68.56
Training cnn
Epoch 1, Loss: 1.90, Accuracy: 23.94, Test Loss: 1.84, Test Accuracy: 31.03
Epoch 30, Loss: 1.40, Accuracy: 76.73, Test Loss: 1.48, Test Accuracy: 68.33
Loading training data: traindata_manual/AB2_traindata_4.p
LDA ---- Clean: 90.49, Noisy: 41.23
AUG ---- Clean: 57.14, Noisy: 46.51
MLP ---- Clean: 59.66, Noisy: 52.14, LDA Clean: 51.51, LDA Noisy: 41.83
MLPB ---- Clean: 86.97, Noisy: 73.43, LDA Clean: 82.20, LDA Noisy: 68.66
CNN ---- Clean: 84.74, Noisy: 74.00, LDA Clean: 85.66, LDA Noisy: 74.17
Loading training data: traindata_manual/AB3_traindata_2.p
Training mlp
Epoch 1, Loss: 1.95, Accuracy: 15.29, Test Loss: 1.94, Test Accuracy: 14.56
Epoch 30, Loss: 1.60, Accuracy: 56.27, Test Loss: 1.61, Test Accuracy: 54.48
Training mlpbeta
Epoch 1, Loss: 1.87, Accuracy: 29.90, Test Loss: 1.81, Test Accuracy: 39.08
Epoch 30, Loss: 1.41, Accuracy: 75.97, Test Loss: 1.50, Test Accuracy: 66.37
Training cnn
Epoch 1, Loss: 1.94, Accuracy: 18.98, Test Loss: 1.94, Test Accuracy: 21.48
Epoch 30, Loss: 1.50, Accuracy: 66.58, Test Loss: 1.59, Test Accuracy: 57.27
Loading training data: traindata_manual/AB3_traindata_4.p
LDA ---- Clean: 76.29, Noisy: 50.60
AUG ---- Clean: 58.46, Noisy: 47.40
MLP ---- Clean: 60.00, Noisy: 58.46, LDA Clean: 58.31, LDA Noisy: 51.09
MLPB ---- Clean: 71.74, Noisy: 72.43, LDA Clean: 59.51, LDA Noisy: 59.46
CNN ---- Clean: 56.77, Noisy: 54.34, LDA Clean: 65.97, LDA Noisy: 58.80
Loading training data: traindata_manual/AB4_traindata_2.p
Training mlp
Epoch 1, Loss: 1.95, Accuracy: 14.43, Test Loss: 1.95, Test Accuracy: 15.02
Epoch 30, Loss: 1.62, Accuracy: 54.47, Test Loss: 1.63, Test Accuracy: 52.00
Training mlpbeta
Epoch 1, Loss: 1.91, Accuracy: 23.23, Test Loss: 1.85, Test Accuracy: 30.92
Epoch 30, Loss: 1.50, Accuracy: 66.51, Test Loss: 1.54, Test Accuracy: 61.73
Training cnn
Epoch 1, Loss: 1.94, Accuracy: 17.33, Test Loss: 1.91, Test Accuracy: 22.13
Epoch 30, Loss: 1.35, Accuracy: 81.49, Test Loss: 1.48, Test Accuracy: 68.05
Loading training data: traindata_manual/AB4_traindata_4.p
LDA ---- Clean: 85.94, Noisy: 52.77
AUG ---- Clean: 53.20, Noisy: 46.23
MLP ---- Clean: 74.80, Noisy: 59.26, LDA Clean: 64.34, LDA Noisy: 50.89
MLPB ---- Clean: 73.54, Noisy: 69.54, LDA Clean: 78.69, LDA Noisy: 71.63
CNN ---- Clean: 82.23, Noisy: 77.91, LDA Clean: 79.97, LDA Noisy: 73.86
Loading training data: traindata_manual/AB5_traindata_2.p
Training mlp
Epoch 1, Loss: 1.95, Accuracy: 14.54, Test Loss: 1.95, Test Accuracy: 13.56
Epoch 30, Loss: 1.63, Accuracy: 53.09, Test Loss: 1.64, Test Accuracy: 51.56
Training mlpbeta
Epoch 1, Loss: 1.94, Accuracy: 17.38, Test Loss: 1.94, Test Accuracy: 15.84
Epoch 30, Loss: 1.48, Accuracy: 68.13, Test Loss: 1.51, Test Accuracy: 65.24
Training cnn
Epoch 1, Loss: 1.94, Accuracy: 18.80, Test Loss: 1.94, Test Accuracy: 17.41
Epoch 30, Loss: 1.49, Accuracy: 67.65, Test Loss: 1.51, Test Accuracy: 64.67
Loading training data: traindata_manual/AB5_traindata_4.p
LDA ---- Clean: 91.09, Noisy: 59.49
AUG ---- Clean: 70.37, Noisy: 47.54
MLP ---- Clean: 77.54, Noisy: 63.63, LDA Clean: 69.51, LDA Noisy: 58.54
MLPB ---- Clean: 79.14, Noisy: 73.23, LDA Clean: 79.26, LDA Noisy: 73.26
CNN ---- Clean: 78.34, Noisy: 72.40, LDA Clean: 75.11, LDA Noisy: 67.00
Loading training data: traindata_manual/AB1_traindata_2.p
Training mlp
Epoch 1, Loss: 1.92, Accuracy: 20.06, Test Loss: 1.86, Test Accuracy: 26.00
Epoch 30, Loss: 1.64, Accuracy: 52.17, Test Loss: 1.79, Test Accuracy: 36.86
Training mlpbeta
Epoch 1, Loss: 1.76, Accuracy: 40.00, Test Loss: 1.73, Test Accuracy: 45.56
Epoch 30, Loss: 1.51, Accuracy: 65.74, Test Loss: 1.57, Test Accuracy: 59.14
Training cnn
Epoch 1, Loss: 1.77, Accuracy: 38.78, Test Loss: 1.73, Test Accuracy: 42.92
Epoch 30, Loss: 1.31, Accuracy: 86.21, Test Loss: 1.43, Test Accuracy: 72.92
no testing data
Loading training data: traindata_manual/AB2_traindata_2.p
Training mlp
Epoch 1, Loss: 1.94, Accuracy: 15.98, Test Loss: 1.94, Test Accuracy: 15.81
Epoch 30, Loss: 1.66, Accuracy: 50.40, Test Loss: 1.68, Test Accuracy: 46.89
Training mlpbeta
Epoch 1, Loss: 1.90, Accuracy: 22.63, Test Loss: 1.86, Test Accuracy: 27.02
Epoch 30, Loss: 1.42, Accuracy: 74.50, Test Loss: 1.47, Test Accuracy: 68.56
Training cnn
Epoch 1, Loss: 1.90, Accuracy: 23.94, Test Loss: 1.84, Test Accuracy: 31.03
Epoch 30, Loss: 1.40, Accuracy: 76.73, Test Loss: 1.48, Test Accuracy: 68.33
Loading training data: traindata_manual/AB2_traindata_4.p
LDA ---- Clean: 90.49, Noisy: 41.23
AUG ---- Clean: 57.14, Noisy: 46.51
MLP ---- Clean: 59.66, Noisy: 52.14, LDA Clean: 51.51, LDA Noisy: 41.83
MLPB ---- Clean: 86.97, Noisy: 73.43, LDA Clean: 82.20, LDA Noisy: 68.66
CNN ---- Clean: 84.74, Noisy: 74.00, LDA Clean: 85.66, LDA Noisy: 74.17
Loading training data: traindata_manual/AB3_traindata_2.p
Training mlp
Epoch 1, Loss: 1.95, Accuracy: 15.29, Test Loss: 1.94, Test Accuracy: 14.56
Epoch 30, Loss: 1.60, Accuracy: 56.27, Test Loss: 1.61, Test Accuracy: 54.48
Training mlpbeta
Epoch 1, Loss: 1.87, Accuracy: 29.90, Test Loss: 1.81, Test Accuracy: 39.08
Epoch 30, Loss: 1.41, Accuracy: 75.97, Test Loss: 1.50, Test Accuracy: 66.37
Training cnn
Epoch 1, Loss: 1.94, Accuracy: 18.98, Test Loss: 1.94, Test Accuracy: 21.48
Epoch 30, Loss: 1.50, Accuracy: 66.58, Test Loss: 1.59, Test Accuracy: 57.27
Loading training data: traindata_manual/AB3_traindata_4.p
LDA ---- Clean: 76.29, Noisy: 50.60
AUG ---- Clean: 58.46, Noisy: 47.40
MLP ---- Clean: 60.00, Noisy: 58.46, LDA Clean: 58.31, LDA Noisy: 51.09
MLPB ---- Clean: 71.74, Noisy: 72.43, LDA Clean: 59.51, LDA Noisy: 59.46
CNN ---- Clean: 56.77, Noisy: 54.34, LDA Clean: 65.97, LDA Noisy: 58.80
Loading training data: traindata_manual/AB4_traindata_2.p
Training mlp
Epoch 1, Loss: 1.95, Accuracy: 14.43, Test Loss: 1.95, Test Accuracy: 15.02
Epoch 30, Loss: 1.62, Accuracy: 54.47, Test Loss: 1.63, Test Accuracy: 52.00
Training mlpbeta
Epoch 1, Loss: 1.91, Accuracy: 23.23, Test Loss: 1.85, Test Accuracy: 30.92
Epoch 30, Loss: 1.50, Accuracy: 66.51, Test Loss: 1.54, Test Accuracy: 61.73
Training cnn
Epoch 1, Loss: 1.94, Accuracy: 17.33, Test Loss: 1.91, Test Accuracy: 22.13
Epoch 30, Loss: 1.35, Accuracy: 81.49, Test Loss: 1.48, Test Accuracy: 68.05
Loading training data: traindata_manual/AB4_traindata_4.p
LDA ---- Clean: 85.94, Noisy: 52.77
AUG ---- Clean: 53.20, Noisy: 46.23
MLP ---- Clean: 74.80, Noisy: 59.26, LDA Clean: 64.34, LDA Noisy: 50.89
MLPB ---- Clean: 73.54, Noisy: 69.54, LDA Clean: 78.69, LDA Noisy: 71.63
CNN ---- Clean: 82.23, Noisy: 77.91, LDA Clean: 79.97, LDA Noisy: 73.86
Loading training data: traindata_manual/AB5_traindata_2.p
Training mlp
Epoch 1, Loss: 1.95, Accuracy: 14.54, Test Loss: 1.95, Test Accuracy: 13.56
Epoch 30, Loss: 1.63, Accuracy: 53.09, Test Loss: 1.64, Test Accuracy: 51.56
Training mlpbeta
Epoch 1, Loss: 1.94, Accuracy: 17.38, Test Loss: 1.94, Test Accuracy: 15.84
Epoch 30, Loss: 1.48, Accuracy: 68.13, Test Loss: 1.51, Test Accuracy: 65.24
Training cnn
Epoch 1, Loss: 1.94, Accuracy: 18.80, Test Loss: 1.94, Test Accuracy: 17.41
Epoch 30, Loss: 1.49, Accuracy: 67.65, Test Loss: 1.51, Test Accuracy: 64.67
Loading training data: traindata_manual/AB5_traindata_4.p
LDA ---- Clean: 91.09, Noisy: 59.49
AUG ---- Clean: 70.37, Noisy: 47.54
MLP ---- Clean: 77.54, Noisy: 63.63, LDA Clean: 69.51, LDA Noisy: 58.54
MLPB ---- Clean: 79.14, Noisy: 73.23, LDA Clean: 79.26, LDA Noisy: 73.26
CNN ---- Clean: 78.34, Noisy: 72.40, LDA Clean: 75.11, LDA Noisy: 67.00
Loading training data: traindata_manual/AB10_traindata_2.p
Training mlp
Epoch 1, Loss: 1.95, Accuracy: 15.08, Test Loss: 1.95, Test Accuracy: 15.48
Epoch 30, Loss: 1.62, Accuracy: 53.96, Test Loss: 1.59, Test Accuracy: 56.59
Training mlpbeta
Epoch 1, Loss: 1.94, Accuracy: 19.71, Test Loss: 1.91, Test Accuracy: 27.05
Epoch 30, Loss: 1.40, Accuracy: 76.52, Test Loss: 1.38, Test Accuracy: 78.08
Training cnn
Epoch 1, Loss: 1.94, Accuracy: 21.37, Test Loss: 1.94, Test Accuracy: 25.35
Epoch 30, Loss: 1.34, Accuracy: 83.16, Test Loss: 1.33, Test Accuracy: 83.27
Loading training data: traindata_manual/AB10_traindata_4.p
LDA ---- Clean: 84.00, Noisy: 42.89
AUG ---- Clean: 61.57, Noisy: 53.54
MLP ---- Clean: 66.63, Noisy: 54.43, LDA Clean: 68.29, LDA Noisy: 58.91
MLPB ---- Clean: 84.51, Noisy: 75.46, LDA Clean: 83.11, LDA Noisy: 75.49
CNN ---- Clean: 82.29, Noisy: 75.00, LDA Clean: 80.94, LDA Noisy: 72.37
Loading training data: traindata_manual/AB11_traindata_2.p
Training mlp
Epoch 1, Loss: 1.95, Accuracy: 14.35, Test Loss: 1.95, Test Accuracy: 15.24
Epoch 30, Loss: 1.60, Accuracy: 56.78, Test Loss: 1.57, Test Accuracy: 58.92
Training mlpbeta
Epoch 1, Loss: 1.94, Accuracy: 18.25, Test Loss: 1.91, Test Accuracy: 20.37
Epoch 30, Loss: 1.37, Accuracy: 80.21, Test Loss: 1.40, Test Accuracy: 76.35
Training cnn
Epoch 1, Loss: 1.94, Accuracy: 20.92, Test Loss: 1.94, Test Accuracy: 23.05
Epoch 30, Loss: 1.51, Accuracy: 65.06, Test Loss: 1.53, Test Accuracy: 63.02
Loading training data: traindata_manual/AB11_traindata_4.p
LDA ---- Clean: 85.69, Noisy: 52.49
AUG ---- Clean: 65.34, Noisy: 50.74
MLP ---- Clean: 72.20, Noisy: 64.11, LDA Clean: 59.74, LDA Noisy: 53.94
MLPB ---- Clean: 82.03, Noisy: 75.29, LDA Clean: 82.29, LDA Noisy: 74.29
CNN ---- Clean: 61.37, Noisy: 57.09, LDA Clean: 71.63, LDA Noisy: 64.26
Loading training data: traindata_manual/AB12_traindata_2.p
Training mlp
Epoch 1, Loss: 1.95, Accuracy: 15.10, Test Loss: 1.95, Test Accuracy: 16.98
Epoch 30, Loss: 1.69, Accuracy: 47.33, Test Loss: 1.71, Test Accuracy: 44.11
Training mlpbeta
Epoch 1, Loss: 1.94, Accuracy: 18.50, Test Loss: 1.94, Test Accuracy: 15.71
Epoch 30, Loss: 1.51, Accuracy: 65.19, Test Loss: 1.60, Test Accuracy: 56.11
Training cnn
Epoch 1, Loss: 1.94, Accuracy: 19.15, Test Loss: 1.94, Test Accuracy: 22.89
Epoch 30, Loss: 1.55, Accuracy: 61.46, Test Loss: 1.66, Test Accuracy: 49.03
Loading training data: traindata_manual/AB12_traindata_4.p
LDA ---- Clean: 54.09, Noisy: 38.23
AUG ---- Clean: 44.00, Noisy: 39.71
MLP ---- Clean: 42.03, Noisy: 40.66, LDA Clean: 47.77, LDA Noisy: 43.97
MLPB ---- Clean: 53.63, Noisy: 50.71, LDA Clean: 50.00, LDA Noisy: 48.49
CNN ---- Clean: 59.43, Noisy: 55.40, LDA Clean: 66.77, LDA Noisy: 58.46
Loading training data: traindata_manual/AB13_traindata_2.p
Training mlp
Epoch 1, Loss: 1.95, Accuracy: 16.41, Test Loss: 1.95, Test Accuracy: 16.95
Epoch 30, Loss: 1.70, Accuracy: 45.85, Test Loss: 1.73, Test Accuracy: 43.14
Training mlpbeta
Epoch 1, Loss: 1.94, Accuracy: 16.49, Test Loss: 1.93, Test Accuracy: 19.30
Epoch 30, Loss: 1.47, Accuracy: 69.38, Test Loss: 1.50, Test Accuracy: 66.06
Training cnn
Epoch 1, Loss: 1.95, Accuracy: 18.84, Test Loss: 1.94, Test Accuracy: 24.41
Epoch 30, Loss: 1.43, Accuracy: 73.38, Test Loss: 1.50, Test Accuracy: 66.51
Loading training data: traindata_manual/AB13_traindata_4.p
LDA ---- Clean: 81.51, Noisy: 42.29
AUG ---- Clean: 41.57, Noisy: 48.40
MLP ---- Clean: 51.49, Noisy: 47.43, LDA Clean: 47.34, LDA Noisy: 45.86
MLPB ---- Clean: 86.57, Noisy: 77.46, LDA Clean: 87.94, LDA Noisy: 76.43
CNN ---- Clean: 67.91, Noisy: 65.91, LDA Clean: 74.46, LDA Noisy: 71.20
Loading training data: traindata_manual/AB14_traindata_2.p
Training mlp
Epoch 1, Loss: 1.95, Accuracy: 15.85, Test Loss: 1.95, Test Accuracy: 15.35
Epoch 30, Loss: 1.54, Accuracy: 62.94, Test Loss: 1.57, Test Accuracy: 59.11
Training mlpbeta
Epoch 1, Loss: 1.89, Accuracy: 24.88, Test Loss: 1.84, Test Accuracy: 29.06
Epoch 30, Loss: 1.46, Accuracy: 70.60, Test Loss: 1.45, Test Accuracy: 71.25
Training cnn
Epoch 1, Loss: 1.94, Accuracy: 24.26, Test Loss: 1.94, Test Accuracy: 28.78
Epoch 30, Loss: 1.37, Accuracy: 79.69, Test Loss: 1.41, Test Accuracy: 75.11
Loading training data: traindata_manual/AB14_traindata_4.p
LDA ---- Clean: 85.49, Noisy: 52.37
AUG ---- Clean: 62.40, Noisy: 58.17
MLP ---- Clean: 64.06, Noisy: 63.43, LDA Clean: 68.03, LDA Noisy: 70.74
MLPB ---- Clean: 76.43, Noisy: 71.29, LDA Clean: 72.71, LDA Noisy: 71.91
CNN ---- Clean: 67.57, Noisy: 66.86, LDA Clean: 64.43, LDA Noisy: 66.77